# **LANL Earthquake Prediction**
### Data used: <a href='https://www.kaggle.com/c/LANL-Earthquake-Prediction'>LANL Earthquake Prediction </a>

# <a id='top'>Nội dung</a>

- <a href='#1'>Mô tả bài toán</a>
- <a href='#2'>Chuẩn bị dữ liệu</a>  
- <a href='#3'>Trích lọc đặc trưng</a>   
- <a href='#4'>Mô tả các đặc trưng</a>
- <a href='#5'>Huấn luyện mô hình</a>
- <a href='#6'>Bagging Regression</a>  
- <a href='#7'>XGBoost</a>
- <a href='#8'>Submission</a>

#### <a href="#top">Back to top</a>
#### <a href="#2">Next</a>
# <a id='1'>Mô tả bài toán</a>
<p style="font-size:20px">Mục tiêu của bài toán là sử dụng sóng địa chấn nhằm dự đoán thời gian xảy ra động đất trong phòng thí nghiệm. Dữ liệu thu được từ những thí nghiệm nổi tiếng nhằm nghiên cứu những tính chất vật lý của động đất. Các nhà nghiên cứu mô phỏng lại các trận động đất trong phòng thí nghiệm với mục đích thu lại dữ liệu về các sóng địa chấn tại mỗi thời điểm và thời gian còn lại đến khi xảy ra động đất tại mỗi thời điểm đó. <span style="background-color: #E9EBEE">acoustic_data</span> là dữ liệu về sóng địa chấn sử dụng để dự đoán thời gian còn lại trước khi trận động đất tiếp theo xảy ra (<span style="background-color: #E9EBEE">time_to_failure</span>).</p>

<p style="font-size:20px">Dữ liệu training là những phần nhỏ (segment), liên tục của dữ liệu thí nghiệm. Dữ liệu test gồm một thư mục chứa nhiều file mỗi file cũng là những segment nhỏ chứa dữ liệu liên tục nhưng không phải là một phần của dữ liệu thí nghiệm. Vì vậy, việc dự đoán không thể giả sử là tuân theo những quy luật thông thường như có thể thấy trong file dữ liệu train.</p>

<p style="font-size:20px">Với mỗi <span style="background-color: #E9EBEE">seg_id</span> trong thư mục test, ta cần dự đoán được <span style="background-color: #E9EBEE">time_to_failure</span> duy nhất tương ứng với thời gian từ thời điểm cùng trong segment cho đến khi động đất xảy ra.</p>

### Mô tả các file dữ liệu
* <p style="font-size:20px"><b>train.csv</b> - Một đoạn liên tục duy nhất lấy từ dữ liệu thí nghiệm.</p>
* <p style="font-size:20px"><b>test</b> - Một thư mục chứa nhiều segment của dữ liệu test với cùng độ dài 150000 mỗi segment.</p>
* <p style="font-size:20px"><b>sample_submission.csv</b> - Mẫu file submission.</p>

### Các trường dữ liệu
* <p style="font-size:20px"><b>acoustic_data</b> - Cường độ sóng địa chấn [int16]</p>
* <p style="font-size:20px"><b>time_to_failure</b> - Thời gian (giây) cho đến khi xảy ra động đất [float64]</p>
* <p style="font-size:20px"><b>seg_id</b> - id của các segment trong tập dữ liệu test</p>

#### <a href="#top">Back to top</a>
#### <a href="#1">Previous</a>
#### <a href="#3">Next</a>
# <a id='2'>Chuẩn bị dữ liệu</a>
### Tải các packages cần thiết
<p style="font-size:15px">Nạp các package cần thiết để sử dụng, trích lọc đặc trưng từ dữ liệu và train model.</p>

In [ ]:
import gc
import os
import time
import logging
import librosa
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from scipy import stats
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold

Không hiển thị các warning messages

In [ ]:
warnings.filterwarnings("ignore")

### Nạp dữ liệu

In [ ]:
test_path = '../input/LANL-Earthquake-Prediction/test'
submission_path = '../input/LANL-Earthquake-Prediction/sample_submission.csv'
train_path = '../input/LANL-Earthquake-Prediction/train.csv'

<p style="font-size:15px"> Đọc dữ liệu từ file train.csv</p>

In [ ]:
%%time
raw_df = pd.read_csv(train_path,
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
raw_df.shape

<p style="font-size:15px"> Lấy mẫu dữ liệu test</p>

In [ ]:
sample_test = pd.read_csv('../input/LANL-Earthquake-Prediction/test/seg_017314.csv')
sample_test.shape

#### <a href="#top">Back to top</a>
#### <a href="#2">Previous</a>
#### <a href="#4">Next</a>
# <a id = '3'>Trích lọc đặc trưng</a>
<p style="font-size:15px">Vì các đoạn trong tập dữ liệu test đều có độ dài 150000 nên dữ liệu train cũng nên được chia thành các phần nhỏ với cùng độ dài 150000</p>

In [ ]:
seg_size = 150000
segments = int(np.floor(raw_df.shape[0] / seg_size))
segments

<p style="font-size:15px">Plot segment đầu tiên và <span style="background-color: #E9EBEE">time_to_failure</span> tương ứng</p>

In [ ]:
sample = raw_df['acoustic_data'][:150000]
ttf = raw_df['time_to_failure'][:150000]

fig, ax1 = plt.subplots(figsize=(16, 8))
plt.title('first segment and time_to_failure')
plt.plot(sample, color='b')
ax1.set_xlabel('training samples')
ax1.set_ylabel('acoustic data')
plt.legend(['acoustic data'], loc=(0.01, 0.95))
ax2 = ax1.twinx()
plt.plot(ttf, color='y')
ax2.set_ylabel('time to failure')
plt.legend(['time to failure'], loc=(0.01, 0.9))
plt.grid(True)

In [ ]:
train_df = pd.DataFrame(index=range(segments), dtype=np.float64)

<p style="font-size:15px">Vì dữ liệu có dạng sóng, nên ta sẽ trích lọc các đặc trưng từ phổ biên độ và trong miền số phức bằng cách sử dụng <i><b>Fast Fourier Transform</b></i> (<span style="background-color: #E9EBEE">np.fft</span>). Với mỗi segment dài 150000, hàm dưới đây sẽ trích lọc ra các đặc trưng thống kê như sau:</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">mean</span> - Giá trị trung bình</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">std</span> - Độ lệch chuẩn</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">max</span> - Giá trị max</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">min</span> - Giá trị min</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">sum</span> - Tổng các phần tử trong segment</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">mad</span> - Mean absolute deviation - Trung bình độ lệch tuyệt đối</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">kurt</span> - Kurtosis (mô tả hình dạng đuôi của phân phối xác suất)</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">skew</span> - Skewness (mô tả tính bất đối xứng của phân phối xác suất)</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">med</span> - Phần tử trung vị</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">abs_mean</span> - Trung bình giá trị tuyệt đối</p>
<p style="font-size:15px"><span style="background-color: #E9EBEE">q95, q99, q05, q01</span> - Phân vị 0.95, 0.99, 0.05 và 0.01</p>

<p style="font-size:15px">Ngoài ra, ứng với mỗi segment, chúng ta sẽ trích lọc đặc trưng MFCC sử dụng thư viện <i><b>librosa</b></i>. Sau đó, ứng với mỗi frame trong MFCC, ta sẽ lấy giá trị trung bình để có được vector MFCC dùng cho việc dự đoán. Việc trích lọc đặc trưng MFCC sử dụng hàm biến đổi Fourier cho thời gian ngắn <i><b>Short Time Fourier Transform</b></i> hay <i><b>STFT</b></i> với các thông số <span style="background-color: #E9EBEE">n_fft</span> = 2048, <span style="background-color: #E9EBEE">hop_length</span> = 512

In [ ]:
def create_features(segment_id, segment, X, targets=True):
    xc = pd.Series(segment['acoustic_data'].values)
    zc = np.fft.fft(xc)
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    
    X.loc[seg_id, 'mean'] = xc.mean()
    X.loc[seg_id, 'std'] = xc.std()
    X.loc[seg_id, 'max'] = xc.max()
    X.loc[seg_id, 'min'] = xc.min()
    X.loc[seg_id, 'sum'] = xc.sum()
    X.loc[seg_id, 'mad'] = xc.mad()
    X.loc[seg_id, 'kurt'] = xc.kurtosis()
    X.loc[seg_id, 'skew'] = xc.skew()
    X.loc[seg_id, 'med'] = xc.median()
    
    X.loc[seg_id, 'q95'] = np.quantile(xc, 0.95)
    X.loc[seg_id, 'q99'] = np.quantile(xc, 0.99)
    X.loc[seg_id, 'q05'] = np.quantile(xc, 0.05)
    X.loc[seg_id, 'q01'] = np.quantile(xc, 0.01)
    
    X.loc[seg_id, 'abs_mean'] = np.abs(xc).mean()
    X.loc[seg_id, 'abs_max'] = np.abs(xc).max()
    X.loc[seg_id, 'abs_min'] = np.abs(xc).min()
    X.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X.loc[seg_id, 'Rmean'] = realFFT.mean()
    X.loc[seg_id, 'Rstd'] = realFFT.std()
    X.loc[seg_id, 'Rmax'] = realFFT.max()
    X.loc[seg_id, 'Rmin'] = realFFT.min()
    
    X.loc[seg_id, 'Imean'] = imagFFT.mean()
    X.loc[seg_id, 'Istd'] = imagFFT.std()
    X.loc[seg_id, 'Imax'] = imagFFT.max()
    X.loc[seg_id, 'Imin'] = imagFFT.min()
    
    X.loc[seg_id, 'std_first_50000'] = xc[:50000].std()
    X.loc[seg_id, 'std_last_50000'] = xc[-50000:].std()
    X.loc[seg_id, 'std_first_25000'] = xc[:25000].std()
    X.loc[seg_id, 'std_last_25000'] = xc[-25000:].std()
    X.loc[seg_id, 'std_first_10000'] = xc[:10000].std()
    X.loc[seg_id, 'std_last_10000'] = xc[-10000:].std()
    
    X.loc[seg_id, 'Rstd_first_50000'] = realFFT[:50000].std()
    X.loc[seg_id, 'Rstd_last_50000'] = realFFT[-50000:].std()
    X.loc[seg_id, 'Rstd_first_25000'] = realFFT[:25000].std()
    X.loc[seg_id, 'Rstd_last_25000'] = realFFT[-25000:].std()
    X.loc[seg_id, 'Rstd_first_10000'] = realFFT[:10000].std()
    X.loc[seg_id, 'Rstd_last_10000'] = realFFT[-10000:].std()
    
    X.loc[seg_id, 'mean_first_50000'] = xc[:50000].mean()
    X.loc[seg_id, 'mean_last_50000'] = xc[-50000:].mean()
    X.loc[seg_id, 'mean_first_25000'] = xc[:25000].mean()
    X.loc[seg_id, 'mean_last_25000'] = xc[-25000:].mean()
    X.loc[seg_id, 'mean_first_10000'] = xc[:10000].mean()
    X.loc[seg_id, 'mean_last_10000'] = xc[-10000:].mean()
    
    X.loc[seg_id, 'Rmean_first_50000'] = realFFT[:50000].mean()
    X.loc[seg_id, 'Rmean_last_50000'] = realFFT[-50000:].mean()
    X.loc[seg_id, 'Rmean_first_25000'] = realFFT[:25000].mean()
    X.loc[seg_id, 'Rmean_last_25000'] = realFFT[-25000:].mean()
    X.loc[seg_id, 'Rmean_first_10000'] = realFFT[:10000].mean()
    X.loc[seg_id, 'Rmean_last_10000'] = realFFT[-10000:].mean()
    
    X.loc[seg_id, 'max_first_50000'] = xc[:50000].max()
    X.loc[seg_id, 'max_last_50000'] = xc[-50000:].max()
    X.loc[seg_id, 'max_first_25000'] = xc[:25000].max()
    X.loc[seg_id, 'max_last_25000'] = xc[-25000:].max()
    X.loc[seg_id, 'max_first_10000'] = xc[:10000].max()
    X.loc[seg_id, 'max_last_10000'] = xc[-10000:].max()
    
    X.loc[seg_id, 'Rmax_first_50000'] = realFFT[:50000].max()
    X.loc[seg_id, 'Rmax_last_50000'] = realFFT[-50000:].max()
    X.loc[seg_id, 'Rmax_first_25000'] = realFFT[:25000].max()
    X.loc[seg_id, 'Rmax_last_25000'] = realFFT[-25000:].max()
    X.loc[seg_id, 'Rmax_first_10000'] = realFFT[:10000].max()
    X.loc[seg_id, 'Rmax_last_10000'] = realFFT[-10000:].max()
    
    X.loc[seg_id, 'min_first_50000'] = xc[:50000].min()
    X.loc[seg_id, 'min_last_50000'] = xc[-50000:].min()
    X.loc[seg_id, 'min_first_25000'] = xc[:25000].min()
    X.loc[seg_id, 'min_last_25000'] = xc[-25000:].min()
    X.loc[seg_id, 'min_first_10000'] = xc[:10000].min()
    X.loc[seg_id, 'min_last_10000'] = xc[-10000:].min()
    
    X.loc[seg_id, 'Rmin_first_50000'] = realFFT[:50000].min()
    X.loc[seg_id, 'Rmin_last_50000'] = realFFT[-50000:].min()
    X.loc[seg_id, 'Rmin_first_25000'] = realFFT[:25000].min()
    X.loc[seg_id, 'Rmin_last_25000'] = realFFT[-25000:].min()
    X.loc[seg_id, 'Rmin_first_10000'] = realFFT[:10000].min()
    X.loc[seg_id, 'Rmin_last_10000'] = realFFT[-10000:].min()
    
    for windows in [5, 10, 50, 100, 500, 1000, 5000, 10000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values
        X.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        X.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
    
    X.loc[seg_id, 'max_to_min_diff'] = xc.max() - np.abs(xc.min())
    
    X.loc[seg_id, 'count_big'] = len(xc[np.abs(xc) > xc.mean()])
    
    mfcc = librosa.feature.mfcc(np.asarray(xc.astype(np.float16)))
    mfcc_mean = mfcc.mean(axis=1)
    for i in range(len(mfcc_mean)):
        X.loc[seg_id, 'mfcc_' + str(i)] = mfcc_mean[i]
        
    if targets:
        X.loc[seg_id, 'time_to_failure'] = segment['time_to_failure'].values[-1]

<p style="font-size:15px">Tính các feature và thêm vào dataframe train_df</span></p>

In [ ]:
for seg_id in tqdm_notebook(range(segments)):
    segment = raw_df.iloc[seg_id*seg_size:seg_id*seg_size+seg_size]
    create_features(seg_id, segment, train_df)

<p style="font-size:15px">Lấy các giá trị (<span style="background-color: #E9EBEE">seg_id</span>) từ file submission mẫu và khởi tạo dataframe <span style="background-color: #E9EBEE">test_df</span></p> 

In [ ]:
submission = pd.read_csv(submission_path, index_col='seg_id')
test_df = pd.DataFrame(columns=train_df.columns, dtype=np.float64, index=submission.index)

<p style="font-size:15px">Do lượng feature lớn nên ta sẽ sử dụng correlation matrix để lựa chọn ra những feature có độ tương quan đối với <span style="background-color: #E9EBEE">time_to_failure</span> cao</p> 

In [ ]:
corrMat = train_df.corr()
corrTarget = abs(corrMat['time_to_failure'])
corrTarget = corrTarget[corrTarget > 0.5]
high_corr_features = corrTarget.index.drop(['time_to_failure'])
high_corr_features

Khởi tạo giá trị cho X_train và y_train

In [ ]:
X_train = train_df[high_corr_features]
y_train = train_df['time_to_failure']
X_train

<p style="font-size:15px">Tính toán các đặc trưng từ dữ liệu test</p>

In [ ]:
for seg_id in tqdm_notebook(test_df.index):
    seg_file = seg_id + '.csv'
    seg = pd.read_csv(os.path.join(test_path, seg_file))
    create_features(seg_id, seg, test_df, targets=False)

Loại bỏ các feature có độ tương quan thấp khỏi test_df

In [ ]:
final_X_test = test_df[high_corr_features]
final_X_test

#### <a href="#top">Back to top</a>
#### <a href="#3">Previous</a>
#### <a href="#5">Next</a>
# <a id='4'>Mô tả các đặc trưng</a>

Hàm bên dưới scatter các điểm dữ liệu theo từng feature và time_to_failure tương ứng

In [ ]:
def plot_feature_scatter(features, X=train_df):
    i = 0
    plt.figure()
    nlines = int(len(features)/2) 
    fig, ax = plt.subplots(nlines, 2, figsize=(20, 5*nlines ))
    for feature in features:
        i+=1
        plt.subplot(nlines,2,i)
        ax[int((i-1)/2)][(i+1) % 2].set_xlabel(feature)
        ax[int((i-1)/2)][(i+1) % 2].set_ylabel('time_to_failure')
        plt.title('{} - time_to_falure correlation)'.format(feature), color='r')
        plt.scatter(x = X[feature], y = X['time_to_failure'])
    plt.show()

In [ ]:
plot_feature_scatter(high_corr_features)

<p style="font-size:15px">Đoạn code dưới sử dụng hàm sns.histplot mô tả histogram và density (mật độ) của các đặc trưng, <span style="color: red">màu đỏ</span> ứng với dữ liệu train và <span style="color: blue">màu xanh</span> ứng với dữ liệu test</p>

In [ ]:
def plot_histplot_features(features, colors=['red', 'blue'], df1=X_train, df2=final_X_test):
    i = 0
    plt.figure()
    nlines = int(len(features)/2)
    fig, ax = plt.subplots(nlines,2,figsize=(20,5*nlines))
    for feature in features:
        i += 1
        plt.subplot(nlines,2,i)
        sns.histplot(df1[feature],color=colors[0], kde=True, bins=40, label='train')
        sns.histplot(df2[feature],color=colors[1], kde=True, bins=40, label='test')
    plt.show()

In [ ]:
plot_histplot_features(high_corr_features)

### Features scaling
<p style="font-size:15px">Đoạn code dưới sẽ scale lại các feature để phù hợp hơn với phân phối chuẩn và sẽ visualize lại bằng các đồ thị mới. Chúng ta sẽ scale với cả dữ liệu train và test. <span style="color: green">Xanh lá</span> ứng với dữ liệu train và <span style="color:blue">xanh dương</span> ứng với dữ liệu test.</p>

In [ ]:
scaler = StandardScaler()
scaler.fit(pd.concat([X_train, final_X_test]))
scaled_X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
scaled_X_test = pd.DataFrame(scaler.transform(final_X_test), columns=final_X_test.columns, index=final_X_test.index)

In [ ]:
plot_histplot_features(high_corr_features, colors=['green', 'blue'], df1=scaled_X_train, df2=scaled_X_test)

### Mô tả các features và time_to_failure
<p style="font-size:15px">Đoạn code dưới mô tả các features và <span style="background-color: #E9EBEE">time_to_failure</span> tương ứng trên cùng một đồ thị.</p>

In [ ]:
def plot_feature_ttf(features):
    i = 0
    plt.figure()
    nlines = int(len(features)/2)
    fig, ax = plt.subplots(nlines,2,figsize=(32,8*nlines))
    for feature in features:
        i += 1
        plt.subplot(nlines,2,i)
        plt.title('({}) and time to failure'.format(feature))
        plt.plot(X_train[feature], color='r')
        ax[int((i-1)/2)][(i+1) % 2].set_xlabel('training samples')
        ax[int((i-1)/2)][(i+1) % 2].set_ylabel('acoustic data ({})'.format(feature), color='r')
        plt.legend(['acoustic data ({})'.format(feature)], loc=(0.01, 0.95))
        ax2 = ax[int((i-1)/2)][(i+1) % 2].twinx()
        plt.plot(y_train, color='b')
        ax2.set_ylabel('time to failure', color='b')
        plt.legend(['time to failure'], loc=(0.01, 0.9))
        plt.grid(True)
    plt.show()

In [ ]:
plot_feature_ttf(high_corr_features)

#### <a href="#top">Back to top</a>
#### <a href="#4">Previous</a>
#### <a href="#6">Next</a>
# <a id='5'>Huấn luyện mô hình</a>

<p style="font-size:15px">Dữ liệu sau khi đã được scale sẽ được đưa vào huấn luyện mô hình SVR để dự đoán. Mô hình được đánh giá dựa vào RMSE và MAE.</p>

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

<p style="font-size:15px">Chia tập dữ liệu train thành dữ liệu train và test</p>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X_train, y_train, test_size=0.33, random_state=42)

Sử dụng *GridSearchCV* để lựa chọn tham số cho mô hình

In [ ]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[0.01, 0.1, 1, 10, 100],'epsilon':[0.1,0.2,0.3,0.4,0.5,0.6]}
# svr = SVR(gamma = 'scale')
# clf = GridSearchCV(svr, parameters)
# clf.fit(X_train,y_train)
# clf.best_params_

Sau khi dùng GridSearchCV thu được bộ tham số: kernel='linear', C=100, epsilon=0.6

In [ ]:
model = SVR(kernel='linear', C=100, epsilon=0.6, gamma='scale')
model.fit(X_train, y_train)

Dự đoán trên tập X_test

In [ ]:
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_pred, y_test)
print('rmse = {} \n mae = {}'.format(rmse,mae))

#### <a href="#top">Back to top</a>
#### <a href="#5">Previous</a>
#### <a href="#7">Next</a>
# <a id='6'>Bagging Regression</a>

Mô tả sai số của các dự đoán

In [ ]:
chart = pd.DataFrame()
chart['target'] = y_test
chart['pred'] = y_pred
chart['off-target'] = abs(chart['target'] - chart['pred'])

In [ ]:
chart.describe()

Có thể thấy bias của kết quả dự đoán tương đối chuẩn tuy nhiên sai số vẫn khá cao. Ta sẽ sử dụng kỹ thuật Bagging cho bài toán regression để làm giảm sai số của mô hình.
Base model được chọn là mô hình SVR(kernel='rbf',gamma='scale',C=1.0,epsilon=0.1)

In [ ]:
model = BaggingRegressor(base_estimator=SVR(), n_estimators=10, random_state=42).fit(X_train, y_train)

Có thể thấy kết quả được cải thiện đôi chút, tuy nhiên không đáng kể

In [ ]:
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_pred, y_test)
print('rmse = {} \n mae = {}'.format(rmse,mae))

#### <a href="#top">Back to top</a>
#### <a href="#6">Previous</a>
#### <a href="#8">Next</a>
# <a id='7'>XGBoost</a>

<p style="font-size:15px">SVR với Bagging cho kết quả tương đối tốt trên tập train. Tuy nhiên khi dùng để dự đoán với dữ liệu test của cuộc thi thì MAE tăng lên đáng kể. Nguyên nhân có thể là do đã xảy ra hiện tượng overfitting. </p>
<p style="font-size:15px">Chúng ta sẽ sử dụng mô hình huấn luyện XGBoost để có thể giảm thiểu hiện tượng overfitting đối với tập dữ liệu train.</p>

In [ ]:
X_train = scaled_X_train
y_train = train_df['time_to_failure']

<p style="font-size:15px">Sử dụng K-fold cross validation với số fold là 5 cho việc huấn luyện và đánh giá mô hình</p>

In [ ]:
%%time
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
 
predictions = np.zeros(len(scaled_X_test))
xgb_params = {'eta': 0.05,
              'max_depth': 10,
              'subsample': 0.8,
              'objective': 'reg:linear',
              'eval_metric': 'mae',
              'nthread': 4}
#run model
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train,y_train.values)):
    X_tr, X_val = X_train.iloc[trn_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[trn_idx], y_train.iloc[val_idx]

    train_data = xgb.DMatrix(data=X_tr, label=y_tr, feature_names=X_train.columns)
    valid_data = xgb.DMatrix(data=X_val, label=y_val, feature_names=X_train.columns)

    watchlist = [(valid_data, 'valid_data')]
    xgb_model = xgb.train(params=xgb_params, dtrain=train_data, num_boost_round=1000, evals=watchlist, early_stopping_rounds=40, verbose_eval=False)

    y_pred = xgb_model.predict(xgb.DMatrix(data=X_val, feature_names=X_tr.columns))
    mae = mean_absolute_error(y_pred, y_val.values)
    print('fold {}: MAE = {}'.format(fold_, mae))
    
    predictions += xgb_model.predict(xgb.DMatrix(scaled_X_test, feature_names=scaled_X_test.columns))/folds.n_splits

#### <a href="#top">Back to top</a>
#### <a href="#7">Previous</a>
# <a id='8'>Submission</a>

XGBoost Submission

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
submission['time_to_failure'] = predictions
submission.to_csv('submission_xgb.csv', index=False)

SVM Bagging Submission

In [ ]:
# submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
# predictions = model.predict(scaled_X_test)
# submission['time_to_failure'] = predictions
# submission.to_csv('submission_svm.csv', index=False)